## Imports
run once

In [1]:
!pip install transformers==4.30.0 --no-deps #compatibility issues between tf and Transformers
!pip install tokenizers==0.13.3 --no-build-isolation
!pip install tensorflow==2.18.0 as
!pip install --upgrade transformers tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 15.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.3
    Uninstalling transformers-4.57.3:
      Successfully uninstalled transformers-4.57.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.9/314.9 kB 13.6 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (pyproject.toml) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
ERROR: Could not find a version that satisfies the requirement as (from versions

In [2]:


import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer, TFRobertaModel
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import os

In [3]:
# Workaround for safetensors issue
os.environ['SAFETENSORS_FAST_GPU'] = '1'
model_name = 'roberta-base'
print(f"\nLoading {model_name}...")

try:
    tokenizer = RobertaTokenizer.from_pretrained(model_name)
    bert_model = TFRobertaModel.from_pretrained(model_name, from_pt=True)
    print("\nModel loaded successfully (from PyTorch checkpoint)")
except Exception as e:
    print(f"Error: {e}")
    print("\nTrying alternative loading method...")
    tokenizer = RobertaTokenizer.from_pretrained(model_name)
    bert_model = TFRobertaModel.from_pretrained(model_name, use_safetensors=False)
    print("\nModel loaded successfully (without safetensors)")
print(f"  Model: {model_name}")
print(f"  Vocab size: {tokenizer.vocab_size:,}")


Loading roberta-base...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint 


Model loaded successfully (from PyTorch checkpoint)
  Model: roberta-base
  Vocab size: 50,265


## Data Upload
unzip them (run once)

In [4]:
from google.colab import drive
import zipfile
import os
drive.mount('/content/drive')

# load datasets
amazon_zip = '/content/drive/MyDrive/571_Proj/amazon_cleaned/finalcombined_cleaned.csv.zip'
yelp_zip = '/content/drive/MyDrive/571_Proj/yelp_cleaned/yelpchi_merged_restaurant_hotel.csv.zip'

with zipfile.ZipFile(amazon_zip, 'r') as zip_ref:
  zip_ref.extractall('content/data/')
  amazon_files = zip_ref.namelist()

with zipfile.ZipFile(yelp_zip, 'r') as zip_ref:
  zip_ref.extractall('content/data/')
  yelp_files = zip_ref.namelist()

print("Datasets extracted")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Datasets extracted


load them as csv's

In [5]:
amazon = pd.read_csv('/content/content/data/finalcombined_cleaned.csv')
yelp = pd.read_csv('/content/content/data/yelpchi_merged_restaurant_hotel.csv')

display data and size

In [6]:
print("AMAZON\n")
print(f"\nAmazon columns: {amazon.columns.tolist()}")
print(amazon.head(5))
print(amazon.shape)

print("YELP\n")
print(f"\nYelp columns: {yelp.columns.tolist()}")
print(yelp.head(5))
print(yelp.shape)


AMAZON


Amazon columns: ['review_id', 'reviewer_id', 'product_id', 'text', 'title', 'rating', 'unixReviewTime', 'label', 'category', 'helpful_votes', 'total_votes', 'helpfulness_ratio']
                  review_id     reviewer_id  product_id  \
0  5a132224741a2384e81d941a  A2GDZUHVGLNDKB  B008BUVXX4   
1  5a132269741a2384e835d60b   AR0I45CCR5HKL  B00HLUAO3U   
2  5a132209741a2384e8142d5f  A2N4C3YLXG0LO0  B005SUHPO6   
3  5a132216741a2384e818b740  A1K6EYQNQ1JEAB  B0077RO08C   
4  5a1321e0741a2384e80658c2  A2TCV4RZUK2LL6  B001YPYYJ8   

                                                text                    title  \
0  this can be used 2 ways. its durable and looks...               great case   
1  excelent item.recomend agood battery compact a...       good cover battery   
2  for the price of this case, its definitely wor...              great value   
3  i bought this because i work 12 hours a day an...  perfect for work or car   
4  this phone works well, has good sound quality,... 

## Data Cleaning
Original Amazon dataset has ~5million rows which is way too big, so we sampled it down.

Can't run this more than once

In [7]:
# we want our amazon dataset to have balanced sampling, and similar to
# the 85/15 real/fake distribution of the yelp dataset

amazon_fake = amazon[amazon['label'] == 1.0]
amazon_real = amazon[amazon['label'] == 0.0]

sample_size = 75000
real = 0.70
fake = 0.30

n_real = int(sample_size * real) #~280k real
n_fake = int(sample_size * fake) #~120k fake

# sample from each class
real_sample = amazon_real.sample(n=n_real, random_state=42)
fake_sample = amazon_fake.sample(n=n_fake, random_state=42)

# combine and shuffle
amazon_sample = pd.concat([real_sample, fake_sample])
amazon_sample = amazon_sample.sample(frac=1, random_state=42) #shuffle


In [8]:
#also need to rename 'text' to 'content' to match Bai(2024) code.
amazon = amazon_sample.rename(columns={'text':'content'})

# only need content and label
amazon = amazon[['content', 'label']].copy()

#verify it worked
print(amazon.shape)
print(amazon.columns.tolist())

#the yelp data also needs a rename
yelp = yelp.rename(columns={'review_text':'content'})
yelp = yelp[['content', 'label']].copy()

#make sure the content (review) is a string
amazon['content'] = amazon['content'].astype(str)
yelp['content'] = yelp['content'].astype(str)

#remove nulls if any
amazon = amazon[amazon['content'].notna()]
yelp = yelp[yelp['content'].notna()]

#verify
print(yelp.shape)
print(yelp.columns.tolist())

(75000, 2)
['content', 'label']
(67395, 2)
['content', 'label']


Now we're minimizing the amount of memory usage for faster processing while still having large enough datasets for good results.

## Train/Val/Test Split
Amazon

In [9]:
# Splitting 70/15/15
train, remaining = train_test_split(
    amazon,
    test_size=0.3,
    random_state=42,
    stratify=amazon['label']
)

valid, test = train_test_split(
    remaining,
    test_size=0.5,
    random_state=42,
    stratify=remaining['label']
)

print("train ", train.shape, " valid ", valid.shape, " test ", test.shape)

#yelp can just stay as yelp since the whole thing is a test set
#printing yelp test just to compare to amazon for sizes

print("yelp test", yelp.shape)

# we also want label distributions to ensure there's an evenish amount
# of fake and real reviews in each set
print(f"train: {train['label'].value_counts().to_dict()}")
print(f"valid: {valid['label'].value_counts().to_dict()}")
print(f"test: {test['label'].value_counts().to_dict()}")
print(f"yelp-test: {yelp['label'].value_counts().to_dict()}")

train  (52500, 2)  valid  (11250, 2)  test  (11250, 2)
yelp test (67395, 2)
train: {0.0: 36750, 1.0: 15750}
valid: {0.0: 7875, 1.0: 3375}
test: {0.0: 7875, 1.0: 3375}
yelp-test: {0: 58476, 1: 8919}


This is a good distribution, with Amazon 70% real ad 30% fake, and Yelp 87% real and 13% fake. To change the distribution of the yelp reviews we'd have to decrease the sample size which is a larger drawback than a slightly more even distribution of fake and real.

## Label Encoding

In [10]:
# This code is adapted from Bai(2024) RoBERTa.py lines 65 - 85

#init encoders
le = LabelEncoder()
oh = OneHotEncoder()

#Amazon labels
train_labels = le.fit_transform(train['label'].values).reshape(-1,1)
valid_labels = le.transform(valid['label'].values).reshape(-1,1)
test_labels = le.transform(test['label'].values).reshape(-1,1)

#One Hot Encoder
train_labels = oh.fit_transform(train_labels).toarray()
valid_labels = oh.transform(valid_labels).toarray()
test_labels = oh.transform(test_labels).toarray()

# Yelp labels
yelp_labels = le.transform(yelp['label'].values).reshape(-1,1)
yelp_labels = oh.transform(yelp_labels).toarray()

print(f"label 0.0 becomes {oh.transform([[0]]).toarray()}")
print(f"label 1.0 becomes {oh.transform([[1]]).toarray()}")

print(train_labels[:3]) #sample to ensure it's looking right


label 0.0 becomes [[1. 0.]]
label 1.0 becomes [[0. 1.]]
[[1. 0.]
 [0. 1.]
 [1. 0.]]


# RoBERTa Time

Code citation: https://github.com/bailuweixi2/RoBERTa-MultiFeature-Model/blob/main/Code/Feature%20Extraction/RoBERTa.py

In [11]:
import tensorflow
from transformers import RobertaTokenizer, TFRobertaModel

# load roberta model and tokenizer
model_name = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(model_name)
bert_model = TFRobertaModel.from_pretrained(
    model_name,
    from_pt=True #force load from pytorch
)

#verify GPU being used
gpus = tensorflow.config.list_physical_devices('GPU')
if gpus:
  print("gpu good")
else:
  print("not using gpu btw")


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


gpu good


This output is good

## Train Amazon - Local
Had to separate because it would crash on saving outputs.
Also had to use chunking for local because it kept timing out.

In [12]:
import tensorflow as tf
import numpy as np
import gc

tf.keras.backend.clear_session()
gc.collect()

print("Extracting Amazon TRAIN - Local chunked")

batch_size = 32
max_length = 128
CHUNK_SIZE = 20000

def encode_and_convert_to_tensors(texts):
    encoded = tokenizer.batch_encode_plus(
        texts,
        padding='max_length',
        max_length=max_length,
        truncation=True,
        return_tensors='tf'
    )
    return encoded

train_texts = train['content'].tolist()
num_train_examples = len(train_texts)

train_all_outputs_0 = []
chunk_num = 0
samples_in_chunk = 0

for i in range(0, num_train_examples, batch_size):
    train_batch_texts = train_texts[i:i + batch_size]
    train_batch_inputs = encode_and_convert_to_tensors(train_batch_texts)
    train_batch_outputs = bert_model(train_batch_inputs)

    train_all_outputs_0.append(train_batch_outputs[0].numpy())
    samples_in_chunk += len(train_batch_texts)

    del train_batch_inputs, train_batch_outputs

    if samples_in_chunk >= CHUNK_SIZE or i + batch_size >= num_train_examples:
        chunk_data = np.concatenate(train_all_outputs_0, axis=0)
        np.save(f'/content/train_local_chunk_{chunk_num}.npy', chunk_data)

        progress = ((i + batch_size) / num_train_examples) * 100
        print(f"  {progress:.1f}% - Chunk {chunk_num} saved")

        del train_all_outputs_0, chunk_data
        train_all_outputs_0 = []
        chunk_num += 1
        samples_in_chunk = 0
        gc.collect()

del train_texts
print("Complete")





Extracting Amazon TRAIN - Local chunked
  38.1% - Chunk 0 saved
  76.2% - Chunk 1 saved
  100.0% - Chunk 2 saved
Complete


## Train Amazon - Global

In [13]:
import tensorflow as tf
import numpy as np
import gc

tf.keras.backend.clear_session()
gc.collect()

print("Extracting Amazon TRAIN - Global")

batch_size = 32
max_length = 128

def encode_and_convert_to_tensors(texts):
    encoded = tokenizer.batch_encode_plus(
        texts,
        padding='max_length',
        max_length=max_length,
        truncation=True,
        return_tensors='tf'
    )
    return encoded

train_texts = train['content'].tolist()
num_train_examples = len(train_texts)

train_all_outputs_1 = []

for i in range(0, num_train_examples, batch_size):
    train_batch_texts = train_texts[i:i + batch_size]
    train_batch_inputs = encode_and_convert_to_tensors(train_batch_texts)
    train_batch_outputs = bert_model(train_batch_inputs)

    train_all_outputs_1.append(train_batch_outputs[1].numpy())

    del train_batch_inputs, train_batch_outputs

    if i % (batch_size * 50) == 0:
        gc.collect()
        progress = (i / num_train_examples) * 100
        print(f"  {progress:.1f}%")

train_global_output = np.concatenate(train_all_outputs_1, axis=0)

print("Saving...")
np.save('/content/train_global_output.npy', train_global_output)

gc.collect()
print("Complete")

Extracting Amazon TRAIN - Global
  0.0%
  3.0%
  6.1%
  9.1%
  12.2%
  15.2%
  18.3%
  21.3%
  24.4%
  27.4%
  30.5%
  33.5%
  36.6%
  39.6%
  42.7%
  45.7%
  48.8%
  51.8%
  54.9%
  57.9%
  61.0%
  64.0%
  67.0%
  70.1%
  73.1%
  76.2%
  79.2%
  82.3%
  85.3%
  88.4%
  91.4%
  94.5%
  97.5%
Saving...
Complete


Had to do concatention separately since it kept timing out

## Not concatenating amazon train, just loading upon training because it keeps causing colab system timeout

## Valid Amazon

In [14]:
import tensorflow as tf
import numpy as np
import gc
import pickle

tf.keras.backend.clear_session()
gc.collect()

print("EXTRACTING AMAZON VAL")

batch_size = 32
max_length = 128

def encode_and_convert_to_tensors(texts):
    encoded = tokenizer.batch_encode_plus(
        texts,
        padding='max_length',
        max_length=max_length,
        truncation=True,
        return_tensors='tf'
    )
    return encoded

valid_texts = valid['content'].tolist()
num_val_examples = len(valid_texts)
print(f"Val samples: {num_val_examples:,}")

# Extract embeddings
val_all_outputs_0 = []
val_all_outputs_1 = []

for i in range(0, num_val_examples, batch_size):
    val_batch_texts = valid_texts[i:i + batch_size]
    val_batch_inputs = encode_and_convert_to_tensors(val_batch_texts)
    val_batch_outputs = bert_model(val_batch_inputs)

    val_all_outputs_0.append(val_batch_outputs[0].numpy())
    val_all_outputs_1.append(val_batch_outputs[1].numpy())

    del val_batch_inputs, val_batch_outputs

    if i % (batch_size * 50) == 0:
        gc.collect()
        progress = (i / num_val_examples) * 100
        print(f"  {progress:.1f}%")

print("100.0% - Extraction complete!")

# SAVE IMMEDIATELY AS PICKLE
print("\nSaving lists directly to disk...")
with open('/content/val_local_list.pkl', 'wb') as f:
    pickle.dump(val_all_outputs_0, f)
print("val_local_list.pkl saved")

with open('/content/val_global_list.pkl', 'wb') as f:
    pickle.dump(val_all_outputs_1, f)
print("val_global_list.pkl saved")

# DELETE FROM MEMORY
del val_all_outputs_0, val_all_outputs_1, valid_texts
gc.collect()

print("\nVAL COMPLETE")

EXTRACTING AMAZON VAL
Val samples: 11,250
  0.0%
  14.2%
  28.4%
  42.7%
  56.9%
  71.1%
  85.3%
  99.6%
100.0% - Extraction complete!

Saving lists directly to disk...
val_local_list.pkl saved
val_global_list.pkl saved

VAL COMPLETE


## Concatenate Valid Amazon

In [15]:
import pickle
import numpy as np

print("Loading")

with open('/content/val_local_list.pkl', 'rb') as f:
    val_local_list = pickle.load(f)

with open('/content/val_global_list.pkl', 'rb') as f:
    val_global_list = pickle.load(f)

print("Concatenating...")
val_local_output = np.concatenate(val_local_list, axis=0)
val_gobal_output = np.concatenate(val_global_list, axis=0)

del val_local_list, val_global_list

print(f"Val local: {val_local_output.shape}")
print(f"Val global: {val_gobal_output.shape}")

# Save
np.save('/content/val_local_output.npy', val_local_output)
np.save('/content/val_gobal_output.npy', val_gobal_output)
print("Saved")

Loading
Concatenating...
Val local: (11250, 128, 768)
Val global: (11250, 768)
Saved


## Test Amazon

In [16]:
import tensorflow as tf
import numpy as np
import gc
import pickle

tf.keras.backend.clear_session()
gc.collect()

print("="*70)
print("EXTRACTING AMAZON TEST")

batch_size = 32
max_length = 128

def encode_and_convert_to_tensors(texts):
    encoded = tokenizer.batch_encode_plus(
        texts,
        padding='max_length',
        max_length=max_length,
        truncation=True,
        return_tensors='tf'
    )
    return encoded

test_texts = test['content'].tolist()
num_test_examples = len(test_texts)
print(f"Test samples: {num_test_examples:,}")

# Extract embeddings
test_all_outputs_0 = []
test_all_outputs_1 = []

for i in range(0, num_test_examples, batch_size):
    test_batch_texts = test_texts[i:i + batch_size]
    test_batch_inputs = encode_and_convert_to_tensors(test_batch_texts)
    test_batch_outputs = bert_model(test_batch_inputs)

    test_all_outputs_0.append(test_batch_outputs[0].numpy())
    test_all_outputs_1.append(test_batch_outputs[1].numpy())

    del test_batch_inputs, test_batch_outputs

    if i % (batch_size * 50) == 0:
        gc.collect()
        progress = (i / num_test_examples) * 100
        print(f"  {progress:.1f}%")

print("100.0% - Extraction complete!")

# SAVE IMMEDIATELY AS PICKLE
print("\nSaving lists directly to disk...")
with open('/content/test_local_list.pkl', 'wb') as f:
    pickle.dump(test_all_outputs_0, f)
print("test_local_list.pkl saved")

with open('/content/test_global_list.pkl', 'wb') as f:
    pickle.dump(test_all_outputs_1, f)
print("test_global_list.pkl saved")

# DELETE FROM MEMORY
del test_all_outputs_0, test_all_outputs_1, test_texts
gc.collect()

print("\nTEST COMPLETE ")

EXTRACTING AMAZON TEST
Test samples: 11,250
  0.0%
  14.2%
  28.4%
  42.7%
  56.9%
  71.1%
  85.3%
  99.6%
100.0% - Extraction complete!

Saving lists directly to disk...
test_local_list.pkl saved
test_global_list.pkl saved

TEST COMPLETE 


## Concatenate Test Amazon

In [17]:
import pickle
import numpy as np

print("Loading")

with open('/content/test_local_list.pkl', 'rb') as f:
    test_local_list = pickle.load(f)

with open('/content/test_global_list.pkl', 'rb') as f:
    test_global_list = pickle.load(f)

print("Concatenating...")
test_local_output = np.concatenate(test_local_list, axis=0)
test_gobal_output = np.concatenate(test_global_list, axis=0)

del test_local_list, test_global_list

print(f"Test local: {test_local_output.shape}")
print(f"Test global: {test_gobal_output.shape}")

# Save
np.save('/content/test_local_output.npy', test_local_output)
np.save('/content/test_gobal_output.npy', test_gobal_output)
print("Saved")

Loading
Concatenating...
Test local: (11250, 128, 768)
Test global: (11250, 768)
Saved


## Yelp

In [18]:
import gc

gc.collect()

0

In [19]:
import tensorflow as tf
import numpy as np
import gc

tf.keras.backend.clear_session()
gc.collect()

print("Extracting Yelp TEST - Chunked")

batch_size = 32
max_length = 128
CHUNK_SIZE = 20000

def encode_and_convert_to_tensors(texts):
    encoded = tokenizer.batch_encode_plus(
        texts,
        padding='max_length',
        max_length=max_length,
        truncation=True,
        return_tensors='tf'
    )
    return encoded

yelp_texts = yelp['content'].tolist()
num_yelp_examples = len(yelp_texts)

yelp_all_outputs_0 = []
yelp_all_outputs_1 = []
chunk_num = 0
samples_in_chunk = 0

for i in range(0, num_yelp_examples, batch_size):
    yelp_batch_texts = yelp_texts[i:i + batch_size]
    yelp_batch_inputs = encode_and_convert_to_tensors(yelp_batch_texts)
    yelp_batch_outputs = bert_model(yelp_batch_inputs)

    yelp_all_outputs_0.append(yelp_batch_outputs[0].numpy())
    yelp_all_outputs_1.append(yelp_batch_outputs[1].numpy())
    samples_in_chunk += len(yelp_batch_texts)

    del yelp_batch_inputs, yelp_batch_outputs

    if samples_in_chunk >= CHUNK_SIZE or i + batch_size >= num_yelp_examples:
        local_chunk = np.concatenate(yelp_all_outputs_0, axis=0)
        global_chunk = np.concatenate(yelp_all_outputs_1, axis=0)

        np.save(f'/content/yelp_local_chunk_{chunk_num}.npy', local_chunk)
        np.save(f'/content/yelp_global_chunk_{chunk_num}.npy', global_chunk)

        progress = ((i + batch_size) / num_yelp_examples) * 100
        print(f"  {progress:.1f}% - Chunk {chunk_num} saved")

        del yelp_all_outputs_0, yelp_all_outputs_1, local_chunk, global_chunk
        yelp_all_outputs_0 = []
        yelp_all_outputs_1 = []
        chunk_num += 1
        samples_in_chunk = 0
        gc.collect()

del yelp_texts
print("Complete")

Extracting Yelp TEST - Chunked
  29.7% - Chunk 0 saved
  59.4% - Chunk 1 saved
  89.0% - Chunk 2 saved
  100.0% - Chunk 3 saved
Complete


In [21]:
np.save('/content/train_labels.npy', train_labels)
np.save('/content/valid_labels.npy', valid_labels)
np.save('/content/test_labels.npy', test_labels)
np.save('/content/yelp_labels.npy', yelp_labels)

### verify files are ready

In [22]:
# Check that all required files exist
import os
import glob

required_files = {
    'Train local chunks': '/content/train_local_chunk_*.npy',
    'Train global': '/content/train_global_output.npy',
    'Val local': '/content/val_local_output.npy',
    'Val global': '/content/val_gobal_output.npy',
    'Test local': '/content/test_local_output.npy',
    'Test global': '/content/test_gobal_output.npy',
    'Yelp local chunks': '/content/yelp_local_chunk_*.npy',
    'Yelp global chunks': '/content/yelp_global_chunk_*.npy',
    'Train labels': '/content/train_labels.npy',
    'Val labels': '/content/valid_labels.npy',
    'Test labels': '/content/test_labels.npy',
    'Yelp labels': '/content/yelp_labels.npy'
}

print("Checking for required files...")
for name, pattern in required_files.items():
    if '*' in pattern:
        files = glob.glob(pattern)
        if files:
            print(f"{name}: Found {len(files)} files")
        else:
            print(f"{name}: NOT FOUND")
    else:
        if os.path.exists(pattern):
            print(f"{name}: Found")
        else:
            print(f"{name}: NOT FOUND")

Checking for required files...
Train local chunks: Found 3 files
Train global: Found
Val local: Found
Val global: Found
Test local: Found
Test global: Found
Yelp local chunks: Found 4 files
Yelp global chunks: Found 4 files
Train labels: Found
Val labels: Found
Test labels: Found
Yelp labels: Found


# Train & Ablate Results

In [ ]:

from google.colab import files
import glob
import os

# Download files one category at a time
print("Downloading files in batches...")

# Batch 1: Labels (small, fast)
print("\n=== Downloading Labels ===")
for f in ['/content/train_labels.npy', '/content/valid_labels.npy',
          '/content/test_labels.npy', '/content/yelp_labels.npy']:
    print(f"Downloading {os.path.basename(f)}...")
    files.download(f)

# Batch 2: Train global (small)
print("\n=== Downloading Train Global ===")
files.download('/content/train_global_output.npy')

# Batch 3: Val/Test (Amazon - medium size)
print("\n=== Downloading Val/Test ===")
for f in ['/content/val_local_output.npy', '/content/val_gobal_output.npy',
          '/content/test_local_output.npy', '/content/test_gobal_output.npy']:
    print(f"Downloading {os.path.basename(f)}...")
    files.download(f)

# Batch 4: Train local chunks (large, one by one)
print("\n=== Downloading Train Local Chunks ===")
train_chunks = sorted(glob.glob('/content/train_local_chunk_*.npy'))
for i, f in enumerate(train_chunks, 1):
    print(f"Downloading chunk {i}/{len(train_chunks)}: {os.path.basename(f)}...")
    files.download(f)

# Batch 5: Yelp chunks (large, one by one)
print("\n=== Downloading Yelp Local Chunks ===")
yelp_local_chunks = sorted(glob.glob('/content/yelp_local_chunk_*.npy'))
for i, f in enumerate(yelp_local_chunks, 1):
    print(f"Downloading chunk {i}/{len(yelp_local_chunks)}: {os.path.basename(f)}...")
    files.download(f)

print("\n=== Downloading Yelp Global Chunks ===")
yelp_global_chunks = sorted(glob.glob('/content/yelp_global_chunk_*.npy'))
for i, f in enumerate(yelp_global_chunks, 1):
    print(f"Downloading chunk {i}/{len(yelp_global_chunks)}: {os.path.basename(f)}...")
    files.download(f)

print("\n✓ ALL DOWNLOADS COMPLETE!")
print("Create a folder called 'fake_review_data' and put all files there")


### Baseline & Ablations

In [1]:
import os
os.makedirs('/content/drive/MyDrive/ablation_results', exist_ok=True)
os.makedirs('/content/drive/MyDrive/ablation_checkpoints', exist_ok=True)

run the cell below every time before running the ours.py script

In [27]:
import gc
import tensorflow as tf

# Clear Keras session
tf.keras.backend.clear_session()

# Delete variables if they exist
try:
    del data, model, train_local_output, train_gobal_output, val_local_output, val_gobal_output
    del test_local_output, test_gobal_output, yelp_local_output, yelp_gobal_output
    del train_labels, val_labels, test_labels, yelp_labels
    del baseline_results
except:
    pass

# Force garbage collection
gc.collect()

print("Memory cleared")

Memory cleared


Also for some reason the yelp labels are flipped?? Doing a quick fix here and will fix properly later

In [5]:
import numpy as np

# Load and flip Yelp labels
yelp_labels = np.load('/content/yelp_labels.npy')
print(f"Original shape: {yelp_labels.shape}")
print(f"Original first 5 rows:\n{yelp_labels[:5]}")

# Flip the columns
yelp_labels_fixed = yelp_labels[:, ::-1]

print(f"\nFlipped first 5 rows:\n{yelp_labels_fixed[:5]}")

# Save the fixed version
np.save('/content/yelp_labels.npy', yelp_labels_fixed)

# Also save to Drive as backup
np.save('/content/drive/MyDrive/yelp_labels_FIXED.npy', yelp_labels_fixed)

Original shape: (67395, 2)
Original first 5 rows:
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]

Flipped first 5 rows:
[[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [6]:
"""
Fake Review Detection: Baseline Training and Feature Ablation Analysis
Adapted from Bai(2024) with some changes made to avoid colab's 90s timeout,
 uses our data, and adapted to do ablation studies.
"""

import numpy as np
import glob
import tensorflow as tf
from keras.layers import *
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
import gc
import json
from datetime import datetime
import os
from pathlib import Path
from tqdm import tqdm

# Mount Google Drive for saving checkpoints and results
from google.colab import drive
drive.mount('/content/drive')

class Config:
    # Data paths - Colab
    TRAIN_LOCAL_CHUNKS = '/content/train_local_chunk_*.npy'
    TRAIN_GLOBAL = '/content/train_global_output.npy'
    VAL_LOCAL = '/content/val_local_output.npy'
    VAL_GLOBAL = '/content/val_gobal_output.npy'
    TEST_LOCAL = '/content/test_local_output.npy'
    TEST_GLOBAL = '/content/test_gobal_output.npy'
    YELP_LOCAL_CHUNKS = '/content/yelp_local_chunk_*.npy'
    YELP_GLOBAL_CHUNKS = '/content/yelp_global_chunk_*.npy'

    # Label paths
    TRAIN_LABELS = '/content/train_labels.npy'
    VAL_LABELS = '/content/valid_labels.npy'
    TEST_LABELS = '/content/test_labels.npy'
    YELP_LABELS = '/content/yelp_labels.npy'

    # Model hyperparameters
    MAX_LENGTH = 128
    EMBEDDING_DIM_1 = 768
    EMBEDDING_DIM_2 = 64
    EXPAND_LENGTH = 12
    NUM_FILTERS = 64
    NUM_CLASSES = 2

    # Training parameters
    EPOCHS = 100
    ES_PATIENCE = 2
    BATCH_SIZE = 128

    # Ablation settings
    ABLATION_FEATURES = ['global', 'local', 'bilstm', 'cnn', 'attention']

    # Output directory - save to Google Drive
    RESULTS_DIR = '/content/drive/MyDrive/ablation_results'
    CHECKPOINT_DIR = '/content/drive/MyDrive/ablation_checkpoints'


class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(
            name="att_weight",
            shape=(input_shape[-1], 1),
            initializer="normal"
        )
        self.b = self.add_weight(
            name="att_bias",
            shape=(input_shape[1], 1),
            initializer="zeros"
        )
        super(AttentionLayer, self).build(input_shape)

    def call(self, inputs):
        et = tf.keras.backend.squeeze(
            tf.keras.backend.tanh(
                tf.keras.backend.dot(inputs, self.W) + self.b
            ),
            axis=-1
        )
        at = tf.keras.backend.softmax(et, axis=-1)
        at = tf.keras.backend.expand_dims(at, axis=-1)
        output = tf.keras.backend.sum(inputs * at, axis=1)
        return output


def check_gpu():
    """Check GPU availability"""
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        print(f"GPU detected: {len(gpus)} device(s)")
        for gpu in gpus:
            print(f"  {gpu}")
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            print("GPU memory growth enabled")
        except RuntimeError as e:
            print(f"GPU setup warning: {e}")
    else:
        print("No GPU detected, using CPU")
    return len(gpus) > 0


def verify_data_files():
    """Check that all required data files exist"""
    print("VERIFYING DATA FILES")

    required_files = {
        'Train local chunks': Config.TRAIN_LOCAL_CHUNKS,
        'Train global': Config.TRAIN_GLOBAL,
        'Val local': Config.VAL_LOCAL,
        'Val global': Config.VAL_GLOBAL,
        'Test local': Config.TEST_LOCAL,
        'Test global': Config.TEST_GLOBAL,
        'Yelp local chunks': Config.YELP_LOCAL_CHUNKS,
        'Yelp global chunks': Config.YELP_GLOBAL_CHUNKS,
        'Train labels': Config.TRAIN_LABELS,
        'Val labels': Config.VAL_LABELS,
        'Test labels': Config.TEST_LABELS,
        'Yelp labels': Config.YELP_LABELS
    }

    all_good = True
    for name, path in required_files.items():
        if '*' in path:
            files = glob.glob(path)
            if files:
                print(f"{name}: Found {len(files)} files")
            else:
                print(f"{name}: NOT FOUND at {path}")
                all_good = False
        else:
            if os.path.exists(path):
                size_mb = os.path.getsize(path) / (1024**2)
                print(f"{name}: Found ({size_mb:.2f} MB)")
            else:
                print(f"{name}: NOT FOUND at {path}")
                all_good = False


    if all_good:
        print("All data files ready")
    else:
        print("ERROR: Some files missing")


    return all_good


def load_data():
    """Load all preprocessed embeddings and labels with progress bars"""

    print("LOADING DATA")


    # Train local
    print("Loading train local chunks")
    train_local_chunks = sorted(glob.glob(Config.TRAIN_LOCAL_CHUNKS))
    train_local_parts = []
    for chunk_file in tqdm(train_local_chunks, desc="Train local"):
        train_local_parts.append(np.load(chunk_file))
    train_local_output = np.concatenate(train_local_parts, axis=0)
    del train_local_parts
    gc.collect()
    print(f"Train local shape: {train_local_output.shape}")

    # Train global
    print("Loading train global")
    train_gobal_output = np.load(Config.TRAIN_GLOBAL)
    print(f"Train global shape: {train_gobal_output.shape}")

    # Validation
    print("Loading validation data")
    val_local_output = np.load(Config.VAL_LOCAL)
    val_gobal_output = np.load(Config.VAL_GLOBAL)
    print(f"Val local shape: {val_local_output.shape}")
    print(f"Val global shape: {val_gobal_output.shape}")

    # Test Amazon
    print("Loading Amazon test data")
    test_local_output = np.load(Config.TEST_LOCAL)
    test_gobal_output = np.load(Config.TEST_GLOBAL)
    print(f"Test local shape: {test_local_output.shape}")
    print(f"Test global shape: {test_gobal_output.shape}")

    # Yelp
    print("Loading Yelp test chunks")
    yelp_local_chunks = sorted(glob.glob(Config.YELP_LOCAL_CHUNKS))
    yelp_global_chunks = sorted(glob.glob(Config.YELP_GLOBAL_CHUNKS))

    yelp_local_parts = []
    for chunk_file in tqdm(yelp_local_chunks, desc="Yelp local"):
        yelp_local_parts.append(np.load(chunk_file))
    yelp_local_output = np.concatenate(yelp_local_parts, axis=0)
    del yelp_local_parts

    yelp_global_parts = []
    for chunk_file in tqdm(yelp_global_chunks, desc="Yelp global"):
        yelp_global_parts.append(np.load(chunk_file))
    yelp_gobal_output = np.concatenate(yelp_global_parts, axis=0)
    del yelp_global_parts

    gc.collect()
    print(f"Yelp local shape: {yelp_local_output.shape}")
    print(f"Yelp global shape: {yelp_gobal_output.shape}")

    # Labels
    print("Loading labels")
    train_labels = np.load(Config.TRAIN_LABELS)
    val_labels = np.load(Config.VAL_LABELS)
    test_labels = np.load(Config.TEST_LABELS)
    yelp_labels = np.load(Config.YELP_LABELS)
    print(f"Train labels shape: {train_labels.shape}")
    print(f"Val labels shape: {val_labels.shape}")
    print(f"Test labels shape: {test_labels.shape}")
    print(f"Yelp labels shape: {yelp_labels.shape}")

    # Reshape global outputs
    print("Reshaping global outputs")
    train_gobal_output = tf.reshape(train_gobal_output, [len(train_labels), Config.EMBEDDING_DIM_2, Config.EXPAND_LENGTH])
    val_gobal_output = tf.reshape(val_gobal_output, [len(val_labels), Config.EMBEDDING_DIM_2, Config.EXPAND_LENGTH])
    test_gobal_output = tf.reshape(test_gobal_output, [len(test_labels), Config.EMBEDDING_DIM_2, Config.EXPAND_LENGTH])
    yelp_gobal_output = tf.reshape(yelp_gobal_output, [len(yelp_labels), Config.EMBEDDING_DIM_2, Config.EXPAND_LENGTH])

    print("DATA LOADING COMPLETE")


    return {
        'train': (train_local_output, train_gobal_output, train_labels),
        'val': (val_local_output, val_gobal_output, val_labels),
        'test': (test_local_output, test_gobal_output, test_labels),
        'yelp': (yelp_local_output, yelp_gobal_output, yelp_labels)
    }


def build_model(ablate_feature=None):
    """Build the RoBERTa-based multi-feature model"""
    print(f"Building model (ablating: {ablate_feature if ablate_feature else 'None'})")

    use_local = ablate_feature != 'local'
    use_global = ablate_feature != 'global'
    use_bilstm = ablate_feature != 'bilstm'
    use_cnn = ablate_feature != 'cnn'
    use_attention = ablate_feature != 'attention'

    inputs = []
    branches = []

    # Local branch
    if use_local:
        input_1 = Input(shape=(Config.MAX_LENGTH, Config.EMBEDDING_DIM_1), name='local_input')
        inputs.append(input_1)

        if use_bilstm:
            x1 = Bidirectional(LSTM(Config.NUM_FILTERS, return_sequences=True))(input_1)
            x1 = Dropout(0.2)(x1)
        else:
            x1 = Flatten()(input_1)
            x1 = Dense(Config.NUM_FILTERS * 2, activation='relu')(x1)
            x1 = Dropout(0.2)(x1)
            x1 = Reshape((1, Config.NUM_FILTERS * 2))(x1)

        if use_attention and use_bilstm:
            x1 = AttentionLayer()(x1)
        elif not use_attention and use_bilstm:
            x1 = GlobalMaxPooling1D()(x1)
        else:
            x1 = Flatten()(x1)

        branches.append(x1)

    # Global branch
    if use_global:
        input_2 = Input(shape=(Config.EMBEDDING_DIM_2, Config.EXPAND_LENGTH), name='global_input')
        inputs.append(input_2)

        if use_cnn:
            x2 = Conv1D(32, 5, activation='relu')(input_2)
            x2 = MaxPool1D(pool_size=2)(x2)
            x2 = Conv1D(64, 6, activation='relu')(x2)
            x2 = MaxPool1D(pool_size=2)(x2)
            x2 = Dropout(0.1)(x2)
        else:
            x2 = Flatten()(input_2)
            x2 = Dense(64, activation='relu')(x2)
            x2 = Dropout(0.1)(x2)
            x2 = Reshape((1, 64))(x2)

        if use_attention and use_cnn:
            x2 = AttentionLayer()(x2)
        elif not use_attention and use_cnn:
            x2 = GlobalMaxPooling1D()(x2)
        else:
            x2 = Flatten()(x2)

        branches.append(x2)

    if len(branches) == 0:
        raise ValueError("Cannot ablate all features")

    # Fusion
    if len(branches) > 1:
        concatenated = Concatenate()(branches)
    else:
        concatenated = branches[0]

    # Classification head
    dense = Dense(32, activation='relu')(concatenated)
    dense = Dropout(0.3)(dense)
    output = Dense(Config.NUM_CLASSES, activation='softmax')(dense)

    model = Model(inputs=inputs, outputs=output)
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    print("Model built successfully")
    return model


def prepare_inputs(local_data, global_data, ablate_feature=None):
    """Prepare model inputs based on ablation configuration"""
    use_local = ablate_feature != 'local'
    use_global = ablate_feature != 'global'

    inputs = []
    if use_local:
        inputs.append(local_data)
    if use_global:
        inputs.append(global_data)

    return inputs


def train_model(model, data, ablate_feature=None, experiment_name='baseline'):
    """Train the model with checkpointing"""
    train_local, train_global, train_labels = data['train']
    val_local, val_global, val_labels = data['val']

    train_inputs = prepare_inputs(train_local, train_global, ablate_feature)
    val_inputs = prepare_inputs(val_local, val_global, ablate_feature)

    print(f"Training model: {experiment_name}")
    print(f"Train samples: {len(train_labels):,}")
    print(f"Val samples: {len(val_labels):,}")

    # Create checkpoint directory
    os.makedirs(Config.CHECKPOINT_DIR, exist_ok=True)
    checkpoint_path = os.path.join(Config.CHECKPOINT_DIR, f'{experiment_name}_epoch_{{epoch:02d}}_val_acc_{{val_accuracy:.4f}}.keras')

    callbacks = [
        EarlyStopping(
            patience=Config.ES_PATIENCE,
            restore_best_weights=True,
            verbose=1
        ),
        ModelCheckpoint(
            filepath=checkpoint_path,
            save_best_only=True,
            monitor='val_accuracy',
            mode='max',
            verbose=1
        )
    ]

    history = model.fit(
        train_inputs,
        train_labels,
        epochs=Config.EPOCHS,
        batch_size=Config.BATCH_SIZE,
        validation_data=(val_inputs, val_labels),
        callbacks=callbacks,
        verbose=1
    )

    print("Training complete")
    return history


def evaluate_model(model, data, dataset_name, ablate_feature=None):
    """Evaluate model on a dataset"""
    local_data, global_data, labels = data
    inputs = prepare_inputs(local_data, global_data, ablate_feature)

    print(f"Evaluating on {dataset_name}")
    loss, accuracy = model.evaluate(inputs, labels, verbose=0)

    predictions = model.predict(inputs, verbose=0)
    predicted_labels = np.argmax(predictions, axis=1)
    true_labels = np.argmax(labels, axis=1)

    from sklearn.metrics import classification_report, confusion_matrix
    report = classification_report(
        true_labels,
        predicted_labels,
        target_names=['Real (0)', 'Fake (1)'],
        digits=5,
        output_dict=True
    )

    cm = confusion_matrix(true_labels, predicted_labels)

    results = {
        'dataset': dataset_name,
        'ablated_feature': ablate_feature if ablate_feature else 'None',
        'loss': float(loss),
        'accuracy': float(accuracy),
        'classification_report': report,
        'confusion_matrix': cm.tolist()
    }

    print(f"{dataset_name} - Loss: {loss:.5f}, Accuracy: {accuracy:.5f}")

    return results


def save_results(results, filename):
    """Save results to JSON file"""
    os.makedirs(Config.RESULTS_DIR, exist_ok=True)
    filepath = os.path.join(Config.RESULTS_DIR, filename)

    with open(filepath, 'w') as f:
        json.dump(results, f, indent=2)

    print(f"Results saved to {filepath}")


def print_summary(all_results):
    """Print summary table"""

    print("EXPERIMENT RESULTS SUMMARY")

    print(f"{'Model':<25} {'Amazon Acc':<15} {'Yelp Acc':<15} {'Impact Score':<15}")


    baseline = None
    for result in all_results:
        if result['ablated_feature'] == 'None':
            baseline = result
            break

    if baseline:
        baseline_yelp_acc = baseline['yelp_accuracy']

        for result in all_results:
            model_name = f"Ablate: {result['ablated_feature']}"
            amazon_acc = result['amazon_accuracy']
            yelp_acc = result['yelp_accuracy']

            if result['ablated_feature'] == 'None':
                impact_score = 0.0
            else:
                impact_score = baseline_yelp_acc - yelp_acc

            print(f"{model_name:<25} {amazon_acc:<15.5f} {yelp_acc:<15.5f} {impact_score:<15.5f}")


    print("Impact Score = Baseline Yelp Accuracy - Ablation Yelp Accuracy")
    print("Higher impact score means feature is more critical for cross-domain transfer")



def run_baseline_experiment(data):
    """Run baseline experiment"""

    print("BASELINE EXPERIMENT - Full Model")


    tf.keras.backend.clear_session()
    gc.collect()

    model = build_model(ablate_feature=None)
    history = train_model(model, data, ablate_feature=None, experiment_name='baseline')

    amazon_results = evaluate_model(model, data['test'], 'Amazon Test', ablate_feature=None)
    yelp_results = evaluate_model(model, data['yelp'], 'Yelp Test', ablate_feature=None)

    model.save(os.path.join(Config.RESULTS_DIR, 'baseline_model.keras'))

    results = {
        'experiment': 'baseline',
        'ablated_feature': 'None',
        'amazon_accuracy': amazon_results['accuracy'],
        'amazon_results': amazon_results,
        'yelp_accuracy': yelp_results['accuracy'],
        'yelp_results': yelp_results,
        'training_history': {
            'loss': [float(x) for x in history.history['loss']],
            'accuracy': [float(x) for x in history.history['accuracy']],
            'val_loss': [float(x) for x in history.history['val_loss']],
            'val_accuracy': [float(x) for x in history.history['val_accuracy']]
        }
    }

    save_results(results, 'baseline_results.json')

    del model
    tf.keras.backend.clear_session()
    gc.collect()

    return results


def run_ablation_experiment(data, feature_to_ablate):
    """Run single ablation experiment"""

    print(f"ABLATION EXPERIMENT - Removing {feature_to_ablate.upper()}")


    tf.keras.backend.clear_session()
    gc.collect()

    model = build_model(ablate_feature=feature_to_ablate)
    history = train_model(model, data, ablate_feature=feature_to_ablate, experiment_name=f'ablation_{feature_to_ablate}')

    amazon_results = evaluate_model(model, data['test'], 'Amazon Test', ablate_feature=feature_to_ablate)
    yelp_results = evaluate_model(model, data['yelp'], 'Yelp Test', ablate_feature=feature_to_ablate)

    model.save(os.path.join(Config.RESULTS_DIR, f'ablation_{feature_to_ablate}_model.keras'))

    results = {
        'experiment': 'ablation',
        'ablated_feature': feature_to_ablate,
        'amazon_accuracy': amazon_results['accuracy'],
        'amazon_results': amazon_results,
        'yelp_accuracy': yelp_results['accuracy'],
        'yelp_results': yelp_results,
        'training_history': {
            'loss': [float(x) for x in history.history['loss']],
            'accuracy': [float(x) for x in history.history['accuracy']],
            'val_loss': [float(x) for x in history.history['val_loss']],
            'val_accuracy': [float(x) for x in history.history['val_accuracy']]
        }
    }

    save_results(results, f'ablation_{feature_to_ablate}_results.json')

    del model
    tf.keras.backend.clear_session()
    gc.collect()

    return results


def main():
    """Main execution function"""

    print("FAKE REVIEW DETECTION: BASELINE AND ABLATION ANALYSIS")
    print("COLAB VERSION")

    print(f"Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


    # Check GPU
    has_gpu = check_gpu()

    # Verify data
    if not verify_data_files():
        print("ERROR: Cannot proceed without data files")
        return

    # Load data
    data = load_data()

    # Run baseline
    baseline_results = run_baseline_experiment(data)

    # Run ablations
    ablation_results = []
    for feature in Config.ABLATION_FEATURES:
        result = run_ablation_experiment(data, feature)
        ablation_results.append(result)

    # Compile all results
    all_results = [baseline_results] + ablation_results
    save_results(all_results, 'all_results.json')

    # Print summary
    print_summary(all_results)


    print("ALL EXPERIMENTS COMPLETE")
    print(f"End time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Results saved to: {Config.RESULTS_DIR}")



if __name__ == "__main__":
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
FAKE REVIEW DETECTION: BASELINE AND ABLATION ANALYSIS
COLAB VERSION
Start time: 2025-12-16 03:12:55
GPU detected: 1 device(s)
  PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
GPU memory growth enabled
VERIFYING DATA FILES
Train local chunks: Found 3 files
Train global: Found (153.81 MB)
Val local: Found (4218.75 MB)
Val global: Found (32.96 MB)
Test local: Found (4218.75 MB)
Test global: Found (32.96 MB)
Yelp local chunks: Found 4 files
Yelp global chunks: Found 4 files
Train labels: Found (0.80 MB)
Val labels: Found (0.17 MB)
Test labels: Found (0.17 MB)
Yelp labels: Found (1.03 MB)
All data files ready
LOADING DATA
Loading train local chunks


Train local: 100%|██████████| 3/3 [01:38<00:00, 32.90s/it]


Train local shape: (52500, 128, 768)
Loading train global
Train global shape: (52500, 768)
Loading validation data
Val local shape: (11250, 128, 768)
Val global shape: (11250, 768)
Loading Amazon test data
Test local shape: (11250, 128, 768)
Test global shape: (11250, 768)
Loading Yelp test chunks


Yelp global: 100%|██████████| 4/4 [00:00<00:00, 53.20it/s]


Yelp local shape: (67395, 128, 768)
Yelp global shape: (67395, 768)
Loading labels
Train labels shape: (52500, 2)
Val labels shape: (11250, 2)
Test labels shape: (11250, 2)
Yelp labels shape: (67395, 2)
Reshaping global outputs
DATA LOADING COMPLETE
BASELINE EXPERIMENT - Full Model
Building model (ablating: None)
Model built successfully
Training model: baseline
Train samples: 52,500
Val samples: 11,250
Epoch 1/100
409/411 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.8396 - loss: 0.3699
Epoch 1: val_accuracy improved from -inf to 0.90622, saving model to /content/drive/MyDrive/ablation_checkpoints/baseline_epoch_01_val_acc_0.9062.keras
411/411 ━━━━━━━━━━━━━━━━━━━━ 22s 45ms/step - accuracy: 0.8400 - loss: 0.3694 - val_accuracy: 0.9062 - val_loss: 0.2321
Epoch 2/100
409/411 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.9156 - loss: 0.2252
Epoch 2: val_accuracy improved from 0.90622 to 0.91271, saving model to /content/drive/MyDrive/ablation_checkpoints/baseline_epoch_02_val_acc_0.91

## Result Verification
We found that not only do impact scores not differ much for each ablation study, but also that they don't even differ much from the baseline accuracy which we thought was strange. Maybe this is because other feautures compensate for the missing one, but the code below will verify that the model architectures for the ablations are acutally different.

In [7]:
# Quick test - check model architectures
import numpy as np
from keras.models import load_model

# Load baseline
baseline = load_model('/content/drive/MyDrive/ablation_results/baseline_model.keras',
                     custom_objects={'AttentionLayer': AttentionLayer})

# Load ablation
ablation_local = load_model('/content/drive/MyDrive/ablation_results/ablation_local_model.keras',
                           custom_objects={'AttentionLayer': AttentionLayer})

print("Baseline inputs:", len(baseline.inputs))
print("Baseline trainable params:", baseline.count_params())

print("\nAblation (no local) inputs:", len(ablation_local.inputs))
print("Ablation (no local) trainable params:", ablation_local.count_params())


Baseline inputs: 2
Baseline trainable params: 447374

Ablation (no local) inputs: 1
Ablation (no local) trainable params: 16526
